In [1]:
import pandas as pd 
import numpy as np
import keras
from PIL import Image
import os
import json
import cv2
import torch

C:\Users\AznNo\AppData\Local\Temp\ipykernel_21972\1438016963.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
directory = "data/dataset_2/0"
images = []

def preprocess_image(image_path, target_size):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    return image

for filename in os.listdir(directory):
    # Check if the file is an image (you might want to improve this check)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Construct the full path to the image file
        filepath = os.path.join(directory, filename)
        image = Image.open(filepath)
        print("Image:", filename)
        print("Image format:", image.format)
        print("Image size:", image.size)
        print("Image mode:", image.mode)
        image = preprocess_image(filepath, target_size=(60,40))
        images.append(image)
        break


# # Path to your JSON file
# json_file_path = "BallSimSample\data.json"
# df = pd.read_json(json_file_path)
# print(df.head())

Image: 0.jpg
Image format: JPEG
Image size: (60, 40)
Image mode: L


In [3]:
def process_json(json_file, img_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    length = len(data) - 1
    # print(length)
    # Extract data for each sample
    img_files = [data[str(i)]["img_file"] for i in range(length)]
    car_positions = [data[str(i)]["car_pos"] for i in range(length)]
    car_velocities = [data[str(i)]["car_vel"] for i in range(length)]
    ball_in_frames = [data[str(i)]["ball_in_frame"] for i in range(length)]
    ball_positions = data["ball_trj"]["pos"]
    ball_velocities = data["ball_trj"]["vel"]
    
    # Load and store images
    images = []
    for i, img_file in enumerate(img_files):
        img_path = os.path.join(img_folder, img_file)
        # print(img_path)
        img = cv2.imread(img_path)
        if img is not None:
            # Convert image to numpy array and normalize
            img = img.astype(np.float32) / 255.0
            images.append(img)
            # print("SUCCESS")
        else:
            print(f"Error loading image: {img_path}")
    
    # Convert images to numpy array
    images = np.array(images)
    
    # Create a dictionary with the extracted data
    sample_data = {
        'images': images,
        'car_positions': car_positions,
        'car_velocities': car_velocities,
        'ball_in_frames': ball_in_frames,
        'ball_positions': ball_positions,
        'ball_velocities': ball_velocities
    }
    
    return sample_data

def create_dataframe_from_folders(json_root_folder, img_folder):
    all_samples_data = []
    
    # Iterate over subfolders in the root JSON folder
    for i, folder in enumerate(os.listdir(json_root_folder)):
        folder_path = os.path.join(json_root_folder, folder)
        # print(folder_path)
        if os.path.isdir(folder_path):
            # Initialize an empty list to hold data for the current sample
            sample_data = process_json(os.path.join(folder_path, 'data.json'), img_folder+str(i))
            
            # Append the data for the current sample to the list of all samples
            all_samples_data.append(sample_data)
    
    # Create DataFrame from the list of dictionaries
    df = pd.DataFrame(all_samples_data)
    return df

# Example usage:
json_root_folder = 'data/dataset_2'
img_folder = 'data/dataset_2/'
df = create_dataframe_from_folders(json_root_folder, img_folder)
df.head()

,images,car_positions,car_velocities,ball_in_frames,ball_positions,ball_velocities
0,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","[[0.0, 0.0], [0.08307482825302567, 0.0], [0.16...","[[0.3946054342018719, 0.0], [0.394605434201871...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[3.2902670151050906, 3.281513633270501, 3.273...","[[-0.042516156118187966, -0.04064097131041608,..."
1,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","[[0.0, 0.0], [0.003258051255889352, 0.0], [0.0...","[[0.015475743465474423, 0.0], [0.0154757434654...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[1.999575218031604, 1.9190488863128043, 1.840...","[[-0.3873870646461205, -0.37761308668247406, -..."
2,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","[[0.0, 0.0], [0.12764228713004042, 0.0], [0.25...","[[0.606300863867692, 0.0], [0.606300863867692,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[3.8095351792024363, 3.8620961947552397, 3.91...","[[0.25190478702627794, 0.24742486072535114, 0...."
3,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","[[0.0, 0.0], [0.05495881276932548, 0.0], [0.10...","[[0.261054360654296, 0.0], [0.261054360654296,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[5.644874189720093, 5.819851451301, 5.9928322...","[[0.8358836023314146, 0.8264003826872065, 0.81..."
4,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...","[[0.0, 0.0], [0.17124159517852222, 0.0], [0.34...","[[0.8133975770979806, 0.0], [0.813397577097980...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[[3.497645605848755, 3.616501635382826, 3.7334...","[[0.5690507228445414, 0.5600815577291292, 0.55..."


In [4]:

# first_image = df['images'][89]
# for i in range(len(first_image)):
#     # Convert the current image to uint8 (required by cv2.imshow())
#     current_image = (first_image[i] * 255).astype('uint8')
#     # Display the current image
#     cv2.imshow(f'Image {i+1}', current_image)
#     # Wait for a key press
#     cv2.waitKey(0)

# # Close all OpenCV windows
# cv2.destroyAllWindows()

In [5]:
from sklearn.model_selection import train_test_split

X = df[['images', 'car_positions', 'car_velocities']]
y = df[['ball_positions', 'ball_velocities']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(X_train['images'][1].shape)



(10, 40, 60, 3)


In [21]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, LSTM, Reshape, TimeDistributed,GlobalMaxPool2D, BatchNormalization, Dropout
from keras.models import Model, Sequential

num_images = 10
image_height = 40
image_width = 60 
num_channels = 3
num_classes = 2

def action_model(shape=(112, 112, 3), nbout=3):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    # model.add(GlobalMaxPool2D())
    return model

# Create the convnet
convnet = action_model((image_height, image_width, num_channels))

# Define inputs
image_input = Input(shape=(num_images,) + convnet.input_shape[1:], name='image_input')
velocity_input = Input(shape=(num_images, 2), name='velocity_input')
position_input = Input(shape=(num_images, 2), name='position_input')

# Apply the convnet to a sequence of images
convnet_sequence = TimeDistributed(convnet)(image_input)

# LSTM layer for image sequences
lstm_images = LSTM(32)(convnet_sequence)

# LSTM layers for velocity and position sequences
lstm_velocity = LSTM(32)(velocity_input)
lstm_position = LSTM(32)(position_input)

# Concatenate features
combined = concatenate([lstm_images, lstm_velocity, lstm_position])

# Dense layers for further processing
dense1 = Dense(128, activation='relu')(combined)
dropout = Dropout(0.5)(dense1)

# Output layers for positions and velocities
output_positions = Dense(10 * 2, activation='linear')(dense1)
reshaped_positions = Reshape((2, 10))(output_positions)

output_velocities = Dense(10 * 2, activation='linear')(dense1)
reshaped_velocities = Reshape((2, 10))(output_velocities)

# Define model
model = Model(inputs=[image_input, velocity_input, position_input], outputs=[reshaped_positions, reshaped_velocities])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 10, 40, 60,  0           []                               
                                 3)]                                                              
                                                                                                  
 time_distributed_3 (TimeDistri  (None, 10, 1920)    94144       ['image_input[0][0]']            
 buted)                                                                                           
                                                                                                  
 velocity_input (InputLayer)    [(None, 10, 2)]      0           []                               
                                                                                            

In [22]:
import tensorflow as tf

images_tf = tf.convert_to_tensor(X_train['images'].tolist())
car_positions_tf = tf.convert_to_tensor(X_train['car_positions'].tolist())
car_velocities_tf = tf.convert_to_tensor(X_train['car_velocities'].tolist())
ball_positions_tf = tf.convert_to_tensor(y_train['ball_positions'].tolist())
ball_velocities_tf = tf.convert_to_tensor(y_train['ball_velocities'].tolist())
# print(images_tf.shape)
model.fit([images_tf, car_positions_tf, car_velocities_tf], [ball_positions_tf, ball_velocities_tf], epochs=300, verbose=1)


Epoch 1/300
3/3 [==============================] - 3s 46ms/step - loss: 6.3270 - reshape_6_loss: 6.1118 - reshape_7_loss: 0.2151 - reshape_6_accuracy: 0.2625 - reshape_7_accuracy: 0.0250
Epoch 2/300
3/3 [==============================] - 0s 51ms/step - loss: 5.6932 - reshape_6_loss: 5.4824 - reshape_7_loss: 0.2108 - reshape_6_accuracy: 0.3000 - reshape_7_accuracy: 0.0125
Epoch 3/300
3/3 [==============================] - 0s 49ms/step - loss: 5.1971 - reshape_6_loss: 4.9815 - reshape_7_loss: 0.2156 - reshape_6_accuracy: 0.2812 - reshape_7_accuracy: 0.0250
Epoch 4/300
3/3 [==============================] - 0s 51ms/step - loss: 4.6919 - reshape_6_loss: 4.4751 - reshape_7_loss: 0.2168 - reshape_6_accuracy: 0.2750 - reshape_7_accuracy: 0.0500
Epoch 5/300
3/3 [==============================] - 0s 49ms/step - loss: 4.1964 - reshape_6_loss: 3.9889 - reshape_7_loss: 0.2076 - reshape_6_accuracy: 0.2500 - reshape_7_accuracy: 0.1000
Epoch 6/300
3/3 [==============================] - 0s 48ms/step -

In [23]:
test_images_tf = tf.convert_to_tensor(X_test['images'].tolist())
test_car_positions_tf = tf.convert_to_tensor(X_test['car_positions'].tolist())
test_car_velocities_tf = tf.convert_to_tensor(X_test['car_velocities'].tolist())
test_ball_positions_tf = tf.convert_to_tensor(y_test['ball_positions'].tolist())
test_ball_velocities_tf = tf.convert_to_tensor(y_test['ball_velocities'].tolist())

output = model.evaluate([test_images_tf, test_car_positions_tf, test_car_velocities_tf], [test_ball_positions_tf, test_ball_velocities_tf])

print(f"Total Loss: ", output[0])
print(f"Ball Pos Loss: ", output[1])
print(f"Ball Vel Loss: ", output[2])
print(f"Ball Pos Acc: ", output[3])
print(f"Ball Vel Acc: ", output[4])

1/1 [==============================] - 1s 739ms/step - loss: 2.7521 - reshape_6_loss: 2.4881 - reshape_7_loss: 0.2639 - reshape_6_accuracy: 0.3000 - reshape_7_accuracy: 0.1250
Total Loss:  2.7520625591278076
Ball Pos Loss:  2.4881443977355957
Ball Vel Loss:  0.2639182209968567
Ball Pos Acc:  0.30000001192092896
Ball Vel Acc:  0.125
